In [ ]:
# NCL stuff:
#import Ngl, Nio
import Nio
#import pyncl
#
import pylab as plt
import numpy
#import scipy
#
import os,sys
import time
#import pathlib

import getpass
import subprocess
import contextlib
#
# write a function to script this process (of extracting a layer from the model data).
#. the data are some indexing of (time, altitude, lon, lat). This script will just focus on pulling
#. a complete altitude layer, and from that only the pressure (['U']) variable. We'll re-dim a little
#. bit, since we're just taking the one variable.
#
# NOTE: for now, let's skip the template file. It's a pain to validate, and may not really save us any time
#. anyway.
#def get_U_from_QBO_layer(layer_index=0, fpathname_src='', fpath_dest='', fname_dest=None,
#                         fpathname_template=None, kt_0=0, kt_max=None):
def copy_U_from_QBO_layer(layer_index=0, fpathname_src='', fpath_dest='', fname_dest=None, batch_size=None,
                         kt_0=0, kt_max=None, verbose=0):


    '''
    #
    # keep it simple for now. This script will basically do one job with only a few options.
    # Variables:
    # @layer_index: layer index to extract
    # @fnamename_src: full path abnd filename of source data file
    # @fpath_dest: output data file path. just the path; we want to allow for dynamic filename construction.
    # @fname_dest: output data filename
    # @fpathnamename_template: Full path an dname of a template file -- an empty (or otherwise intialized)
    #. file with the right dimensinos, etc.
    #.  writing the empty file seems to take a long, long time, so this is probably a reasonable way to save time.
    # @kt_0, @kt_max: start and stop time indices. They will default to 0, and {max}
    # 
    ''' 
    #
    if fpathname_src is None:
        fpathname_in = "U_V_T_Z3_plWACCMSC_CTL_122.cam.h2.0001-0202.nc"
    if fpath_out is None:
        fpath_out = ''
    #
    # fname: we'll handle it later... Same for fname_template, kt_max
    #
    # fname_out template:
    # filo = "U_10_QBO_" + dum + ".nc"
    # (but we need to know more before we can )
    #
#     # try to validate the file. Just wrap this whole thing in a try, except. if we get an exception, 
#     #. we just (re-) create the file from scratch.
#     with contextlib.closing(Nio.open_file(fpathname_src, 'r')) as fin:
#         if not fpathname_template is None:
#             try:
#                 with contextlib.closing(Nio.open_file(fpathname_template), 'r') as f_temp:
#                     #
#                     # do we have a 'U' variable? We'll check the size. if the variable does not exist,
#                     #. we'll throw an excetion anyway.
#                     if numpy.shape(fin.variables['U']) != numpy.shape(f_temp.variables['U']):
#                         raise Exception('Shapes of variables[U] are not consistent')
#                     #
#                     # there are probably more efficient ways to do this, but this is logically very solid.
#                     #for ky,val in f_temp.dimensions.items():
#                     for ky in f_temp.variables['U'].dimensions:
#                         # check all dimensions, or just the ones we need?
#                         if not fin.dimensions[ky] == f_temp.dimensikons[ky]:
#                         #if not (ky in fin.dimensions.keys() and fin.dimensions[ky]==val):
#                             raise Exception('Dimensions of output template do not match input')
#                         #
#             except:
#                 print('Excepting on template file validation')
#                 fpathname_template = None
#                 #
#             #
    # create the output file:
    # 
    with contextlib.closing(Nio.open_file(input_file_path, 'r')) as fin:
        # time dimension size:
        # we can get the full dimension size, if we are going to write the whole lot;
        # otherwise, maybe just a subset?
        n_time = fin.dimensions['time']
        if batch_size is None:
            batch_size=int(n_time/50)
        #
        if fname_dest is None or fname_dest='':
            fname_dest = 'U_{}_QBO_k{}.nc'.format(fin.variables['lev_p'][layer_index], layer_index)
        #
        os.system('rm {}'.format(output_file_path))
        if verbose: print('** DEBUG: open output file...')
        with contextlib.closing(Nio.open_file(output_file_path, 'c')) as fout:
            #
            fout.create_dimension('time', n_time)
            fout.create_dimension('lat', fin.dimensions['lat'])
            fout.create_dimension('lon', fin.dimensions['lon'])
            if verbose: print('** DEBUG: dimensions created: {}'.format(time.time()-t0))
            #
            # set the dimension variable values as well:
            # NOTE: this step may not be necessary; these variables might be brought along by
            #. the principal data, since those data have lat, lon, time listed as their dimensions.
            for v_name in ('lat', 'lon', 'time'):
                fout.create_variable(v_name, 'f', (v_name, ) )
                fout.variables[v_name][:] = fin.variables[v_name]
                #
                if verbose: print('** DEBUG: var {} craeted'.format(v_name))
            if verbose: print('** DEBUG: variables created (cumulative time): {}'.format(time.time()-t0))
            #
            fout.create_variable('U','f',('time','lat','lon'))
            setattr(fout.variables['U'], 'standard_name', 'pressure')
            setattr(fout.variables['U'], 'units', 'kPa')
            if verbose: ('** DEBUG: variagble[U] created (cum time): {}'.format(time.time()-t0))
            #
            # NOTE: the .set_value() syntax is required for scaler, non-indexed values (or so I have read).
            #. we can also use it for arrays, but only if we are assigning the whole value -- in other words, the 
            #. dimensinos must match. (this works if dim(udum)==dim(fout)). I assume the exception is if the output
            #. dimension is undefined.
            #
            #fout.variables['u'].assign_value(udum)        # this works if dimensions align
            #fout.variables['u'][0:len(udum)] = udum      # use this for partial assignment (note, i think this does allow 
            #                                            # (aka, will expand) for overflow -- [k,j] > len(fout.variable) )
            #
            if verbose: print('file created: {}'.format(time.time()-t0))
            #
            # now write file:
            for j,k in enumerate(range(0, fin.dimensions['time'], batch_sizee)):
                #print('** ** [{}:{}]'.format(k, k+batch_size_write))
                #
                # remember, our read-data are like:
                # udum = fin.variables['U'][0:batch_size, 2,:,:]
                # and we're making out output file more or less a mirror of that:
                # note: we never explicilty store the data in a local variable; the memory footprint
                #. is determined entirely by the NetCDF class, so really we can probably do this in one batch...
                #  unless we want to parallelize, in which case we can run each of thsed batches an a Process() thread,
                # in a Queue(), Pool(), etc.
                t1=t2
                t2 = time.time()
                k1 = min(k+batch_size, n_time)
                #
                if verbose: print('begin k={}/{} batches [{}:{}]/{} :: {}.'.format(j, n_batches, k, k1, n_time, t2-t1))
                #
                #print('*** k:k1: {}:{}'.format(k,k1))
                fout.variables['U'][k:k1] = fin.variables['U'][k:k1, 2,:,:]
            #
        #


In [ ]:
fname = 'test_file.nc'
os.system('rm {}'.format(fname))
with contextlib.closing(Nio.open_file(fname, 'c')) as fout:
    fout.create_dimension('lat', 100)
    fout.create_dimension('lon', 100)
    #
    fout.create_variable('U', 'd', ('lat', 'lon'))
#
with contextlib.closing(Nio.open_file(fname,'r')) as fin:
    print('** dims: ', fin.dimensions)
    #
with contextlib.closing(Nio.open_file(fname, 'r+')) as fout:
    #
    fout.dimensions.update({'lon':200})
    #
with contextlib.closing(Nio.open_file(fname,'r')) as fin:
    print('** dims: ', fin.dimensions)

In [ ]:
d = {'a':1, 'b':2}


In [ ]:
fin = Nio.open_file(fname,'r')

print(fin.variables.keys())
print(fin.variables['U'].dimensions)
print('size: ', numpy.shape(fin.variables['U']))
fin.close()

In [ ]:
with contextlib.closing(Nio.open_file('/scratch/yoder/U_V_T_Z3_plWACCMSC_CTL_122.cam.h2.0001-0202.nc', 'r')) as fin:
    print('** ', fin.variables.keys())
    print('** lev_p: ', [x for x in fin.variables['lev_p']])
    print('** lev_p[2]: ', fin.variables['lev_p'][2])
    #